In [1]:
%load_ext autoreload
%load_ext autotime

time: 343 µs (started: 2023-03-08 09:15:40 -08:00)


In [2]:
%run init.ipynb

/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


{'cls': <class 'distributed.scheduler.Scheduler'>, 'options': {'protocol': 'tcp://', 'interface': None, 'host': None, 'dashboard_address': ':8787', 'security': None}}
time: 2.44 s (started: 2023-03-08 09:15:40 -08:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 16.1 ms (started: 2023-03-08 09:15:42 -08:00)


In [4]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

time: 548 ms (started: 2023-03-08 09:15:42 -08:00)


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.130.70:8787/status,
Dashboard: http://192.168.130.70:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.130.70:33467,Workers: 0
Dashboard: http://192.168.130.70:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 45.6 ms (started: 2023-03-08 09:15:43 -08:00)


In [6]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 755 µs (started: 2023-03-08 09:15:43 -08:00)


In [8]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=3840,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 877 ms (started: 2023-03-08 09:15:49 -08:00)


In [9]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,1,0,-2127943504,2,0,0.178566,2.712372,MPI_Comm_dup,0,...,app1,,UNKNOWN,0,0,0.0,2.533806e+00,14454688,0,0
1,2,1,0,-2127943504,2,0,2.712592,2.712593,MPI_Comm_rank,0,...,app1,,UNKNOWN,0,0,0.0,9.000000e-07,27125922,0,0
2,3,1,0,-2127943504,2,0,2.712597,2.712597,MPI_Comm_size,0,...,app1,,UNKNOWN,0,0,0.0,0.000000e+00,27125968,0,0
3,4,1,0,-2127943504,2,0,2.712606,2.713452,MPI_Comm_split_type,0,...,app1,,UNKNOWN,0,0,0.0,8.459000e-04,27130287,0,0
4,5,1,0,-2127943504,2,0,2.713541,2.713541,MPI_Comm_rank,0,...,app1,,UNKNOWN,0,0,0.0,0.000000e+00,27135408,0,0


time: 12.3 s (started: 2023-03-08 09:15:50 -08:00)


In [10]:
ddf.index.count().compute()

3981790129

time: 1min 7s (started: 2023-03-08 09:16:03 -08:00)


In [11]:
ddf['tend'].max().compute()

194355120.0

time: 5.22 s (started: 2023-03-08 09:17:10 -08:00)


In [12]:
ddf['file_id'].unique().count().compute()

5

time: 4.2 s (started: 2023-03-08 09:17:15 -08:00)


In [13]:
ddf['proc_id'].unique().count().compute()

5249

time: 4.27 s (started: 2023-03-08 09:17:20 -08:00)


In [14]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmid_min, tmid_max

(0, 1943551213547325)

time: 5.05 ms (started: 2023-03-08 09:17:24 -08:00)


In [15]:
tmid_max / 1e7 / 60

3239252.022578875

time: 3.52 ms (started: 2023-03-08 09:17:24 -08:00)


In [16]:
trange = np.arange(tmid_min, tmid_max, 1e7 * 60)
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 849 ms (started: 2023-03-08 09:17:24 -08:00)


In [17]:
VIEW_TYPES = ['trange', 'file_id', 'proc_id']

time: 544 µs (started: 2023-03-08 09:17:25 -08:00)


In [18]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(ddf: dd.DataFrame, delta: float, max_io_time: float, metric='duration'):
    def set_delta(df: pd.DataFrame):
        df['duration_csp'] = df['duration_sum'].cumsum() / max_io_time
        df['duration_delta'] = df['duration_csp'].diff().fillna(df['duration_csp'])
        df['duration_score'] = np.digitize(df['duration_delta'], bins=DELTA_BINS, right=True)
        df['duration_cut'] = np.choose(df['duration_score'] - 1, choices=DELTA_BINS, mode='clip')
        return df
    ddf = ddf.map_partitions(set_delta)
    return ddf[ddf['duration_delta'] > delta]

time: 1.67 ms (started: 2023-03-08 09:17:25 -08:00)


In [19]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')

main_view = ddf[ddf['cat'] == 0] \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        'file_name': ['first'],
        'proc_name': ['first']
#         'func_id': [unique_flatten()]
    }) \
    .persist()

main_view


Dask DataFrame Structure:
              duration  index   size               file_name proc_name
                   sum  count    min    max    sum     first     first
npartitions=1                                                         
               float32  int64  int64  int64  int64    object    object
                   ...    ...    ...    ...    ...       ...       ...
Dask Name: aggregate-agg, 1 tasks

time: 1.34 s (started: 2023-03-08 09:17:25 -08:00)


In [20]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-9d43a5df8e14cae019b46e5221004457', 0)>}, not_done=set())

time: 1min 57s (started: 2023-03-08 09:17:26 -08:00)


In [21]:
main_view = main_view.reset_index()
main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]
main_view = main_view.persist()

time: 31.1 ms (started: 2023-03-08 09:19:23 -08:00)


In [22]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('rename-4937cd134a0a05336d8a01bbb8b4c10f', 0)>}, not_done=set())

time: 7.99 ms (started: 2023-03-08 09:19:24 -08:00)


In [23]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])


main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
#main_view['metadata_count'] = main_view['metadata_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)


final_view = main_view \
    .groupby(['trange', 'file_id', 'proc_id']) \
    .agg({
        'duration_sum': sum,
        'index_count': sum,
        'size_sum': sum,
        'read_count': sum,
        'write_count': sum,
        'metadata_count': sum,
        'read_size': sum,
        'write_size': sum,
        'metadata_size': sum,
        'read_time': sum,
        'write_time': sum,
        'metadata_time': sum,
        'sequential_count': sum,
        'random_count': sum,
        'sequential_size': sum,
        'random_size': sum,
        'sequential_time': sum,
        'random_time': sum,
        'data_count': sum,
        'data_size': sum,
        'data_time': sum,
#         'workload_type': sum,
        'size_min': min,
        'size_max': max,
        'file_name_first': 'first',
        'proc_name_first': 'first'
    }) \
    .persist()


time: 343 ms (started: 2023-03-08 09:19:24 -08:00)


In [24]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-963f2c3fc312f835bcd90cabbb81811f', 0)>}, not_done=set())

time: 89.7 ms (started: 2023-03-08 09:19:24 -08:00)


In [25]:
final_view = final_view \
    .reset_index() \
    .rename(columns={
        'file_name_first': 'file_name', 
        'proc_name_first': 'proc_name'
    }) \
    .persist()

time: 30.1 ms (started: 2023-03-08 09:19:24 -08:00)


In [26]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('rename-9e5e6384465a0f6eff9c57d2ea4b2fe8', 0)>}, not_done=set())

time: 10.6 ms (started: 2023-03-08 09:19:24 -08:00)


In [27]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_size,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name
0,1,113104469769545699,-929352282003194517,0.000045,45,0,0,0,45,0,...,0,0.000045,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189283297456
1,1,113104469769545699,-929352282003194452,0.000048,47,0,0,0,47,0,...,0,0.000048,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189287557296
2,1,113104469769545699,-929352282003194387,0.000046,49,0,0,0,49,0,...,0,0.000046,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189291817136
3,1,113104469769545699,-929352282003194322,0.000039,51,0,0,0,51,0,...,0,0.000039,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189296076976
4,1,113104469769545699,-929352282003194257,0.000065,53,0,0,0,53,0,...,0,0.000065,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189300336816


time: 44.4 ms (started: 2023-03-08 09:19:24 -08:00)


In [28]:
del main_view

time: 768 µs (started: 2023-03-08 09:19:24 -08:00)


In [29]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_size,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name
0,1,113104469769545699,-929352282003194517,0.000045,45,0,0,0,45,0,...,0,0.000045,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189283297456
1,1,113104469769545699,-929352282003194452,0.000048,47,0,0,0,47,0,...,0,0.000048,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189287557296
2,1,113104469769545699,-929352282003194387,0.000046,49,0,0,0,49,0,...,0,0.000046,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189291817136
3,1,113104469769545699,-929352282003194322,0.000039,51,0,0,0,51,0,...,0,0.000039,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189296076976
4,1,113104469769545699,-929352282003194257,0.000065,53,0,0,0,53,0,...,0,0.000065,0,0,0,0.0,0,0,UNKNOWN,app1#localhost#0#35189300336816


time: 38.5 ms (started: 2023-03-08 09:19:24 -08:00)


In [30]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(view_type: str, ddf: dd.DataFrame, delta: float, metric: str, max_io_time: dd.core.Scalar):
    metric_col, csp_col, delta_col, score_col, cut_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
        f"{metric}_score",
        f"{metric}_cut"
    )

    def set_delta(df: pd.DataFrame):
        df[csp_col] = df[metric_col].cumsum() / max_io_time
        df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
        df[score_col] = np.digitize(df[delta_col], bins=DELTA_BINS, right=True)
        df[cut_col] = np.choose(df[score_col] - 1, choices=DELTA_BINS, mode='clip')
        return df

    index = pd.MultiIndex(levels=[[1]], codes=[[]], names=[view_type])
    meta = dd.utils.make_meta(([
        (metric_col, np.float64),
        (csp_col, np.float64),
        (delta_col, np.float64),
        (score_col, np.int8),
        (cut_col, np.float16),
    ]), index=index)

#     ddf = ddf.map_partitions(set_delta, meta=meta)
    ddf = ddf.map_partitions(set_delta)

    return ddf[ddf[delta_col] > delta]


time: 1.95 ms (started: 2023-03-08 09:19:24 -08:00)


In [31]:
max_io_time = final_view.groupby(['proc_id']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 21 ms (started: 2023-03-08 09:19:24 -08:00)


In [32]:
DELTA = 0.0001
CUT = 0.5

time: 1.41 ms (started: 2023-03-08 09:19:24 -08:00)


In [35]:
import itertools as it

def _set_metric_percentages(ddf: dd.DataFrame, max_io_time: dd.core.Scalar, metric: str):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    ddf[pero_col] = ddf[metric_col] / max_io_time
    ddf[perr_col] = ddf[metric_col] / ddf[metric_col].sum()
    return ddf

def _compute_expanded_view(
    parent_view: dd.DataFrame,
    view_type: str,
    cols: tuple,
    metric: str,
    delta: float,
    max_io_time: dd.core.Scalar
):
    # Get column names
    metric_col, score_col, cut_col = cols
    # Check view type
    if view_type is not 'proc_id':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_id']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Filter view
    filtered_view = filter_delta(
        view_type=view_type,
        ddf=group_view,
        delta=delta,
        metric=metric,
        max_io_time=max_io_time
    )
    # Get score view
    score_view = filtered_view.reset_index()[[view_type, score_col, cut_col]]
    # Find filtered records and set duration scores
    expanded_view = parent_view \
        .query(f"{view_type} in @indices", local_dict={'indices': filtered_view.index.unique()}) \
        .drop(columns=[score_col, cut_col], errors='ignore') \
        .merge(score_view, on=[view_type])    
    # Set metric percentages
    expanded_view = _set_metric_percentages(ddf=expanded_view, max_io_time=max_io_time, metric=metric)
    # Return expanded view
    return expanded_view

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type]['expanded_view'] if parent_type in views else final_view
    # Create colum names
    metric_col, score_col, cut_col = f"{metric}_sum", f"{metric}_score", f"{metric}_cut"
    # Compute expanded view
    expanded_view = _compute_expanded_view(
        parent_view=parent_view,
        view_type=view_type,
        cols=(metric_col, score_col, cut_col),
        metric=metric,
        delta=DELTA,
        max_io_time=max_io_time
    )
    # Compute bottleneck view
    bottleneck_view = expanded_view[expanded_view[cut_col] >= CUT]
    # Set duration scores to expanded view
#     bottleneck_view = filter_delta(
#         view_type=view_type,
#         ddf=bottleneck_view,
#         delta=0,
#         metric=metric,
#         max_io_time=max_io_time
#     )
    # Return views
    views[view_permutation] = {}
    views[view_permutation]['expanded_view'] = expanded_view
    views[view_permutation]['bottleneck_view'] = bottleneck_view

len(views)

15

time: 1.83 s (started: 2023-03-08 09:20:39 -08:00)


In [36]:
for view_key, view_dict in views.items():
    print(view_key, 'expanded_view', len(view_dict['expanded_view']), 'bottleneck_view', len(view_dict['bottleneck_view']))

('trange',) expanded_view 5760 bottleneck_view 128
('file_id',) expanded_view 5632 bottleneck_view 256
('proc_id',) expanded_view 640 bottleneck_view 230
('trange', 'file_id') expanded_view 5632 bottleneck_view 256
('trange', 'proc_id') expanded_view 640 bottleneck_view 230
('file_id', 'trange') expanded_view 5632 bottleneck_view 128
('file_id', 'proc_id') expanded_view 512 bottleneck_view 184
('proc_id', 'trange') expanded_view 640 bottleneck_view 128
('proc_id', 'file_id') expanded_view 512 bottleneck_view 256
('trange', 'file_id', 'proc_id') expanded_view 512 bottleneck_view 184
('trange', 'proc_id', 'file_id') expanded_view 512 bottleneck_view 256
('file_id', 'trange', 'proc_id') expanded_view 512 bottleneck_view 184
('file_id', 'proc_id', 'trange') expanded_view 512 bottleneck_view 128
('proc_id', 'trange', 'file_id') expanded_view 512 bottleneck_view 256
('proc_id', 'file_id', 'trange') expanded_view 512 bottleneck_view 128
time: 8.74 s (started: 2023-03-08 09:20:41 -08:00)


In [37]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_id=('file_name', 'proc_name', 'trange'),
    proc_id=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_id='file_name',
    proc_id='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    ll_view: pd.DataFrame,
    ml_view: pd.DataFrame,
    hl_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = ll_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = hl_view.loc[hl_id]
        ml_row = ml_view.loc[(hl_id, ml_id)]
        ll_row = ll_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view_dict in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view_dict=view_dict,
            view_types=view_types,
        )
    # Generate bottlenecks
#     bottlenecks = _process_bottleneck_views(bottleneck_views=bottleneck_views)
    # Return bottleneck views
    return bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, pd.DataFrame]):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        bottlenecks_delayed.append(_process_bottleneck_view(
            view_key=view_key,
            ll_view=view_dict['low_level_view'],
            ml_view=view_dict['mid_level_view'],
            hl_view=view_dict['high_level_view']
        ))
    # Compute all bottlenecks
    futures = compute(*bottlenecks_delayed, sync=False)
    results = get_client().gather(list(futures))
    for view_key, result in results:
        bottlenecks[view_key] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
        view_key: tuple,
        view_dict: Dict[str, dd.DataFrame],
        view_types: list
    ):
        # Get view type
        view_type = view_key[-1]
        bottleneck_type = BOTTLENECK_TYPE[view_type]

        # Get parent view
        bottleneck_view = view_dict['bottleneck_view']

        # Create lower level view
        low_level_view = bottleneck_view \
            .groupby(list(BOTTLENECK_ORDER[view_type])) \
            .first() \
            .drop(columns=['acc_pat', 'io_cat'], errors='ignore')

        # Non-proc agg columns
        non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
        proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

        if bottleneck_type is not 'proc_name':

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'proc_name']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(proc_agg_dict)

        else:

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'trange']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(non_proc_agg_dict)

        return dict(
            low_level_view=low_level_view.persist(),
            mid_level_view=mid_level_view.persist(),
            high_level_view=high_level_view.persist()
        )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    agg_dict['duration_cut'] = 'first'
    agg_dict['duration_score'] = 'first'
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 7.17 ms (started: 2023-03-08 09:20:53 -08:00)


In [38]:
bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottleneck_views)

15

time: 3.42 s (started: 2023-03-08 09:20:54 -08:00)


In [39]:
for view_key, bottleneck_view_dict in bottleneck_views.items():
    print(view_key, 'hl_view', len(bottleneck_view_dict['high_level_view']), 'ml_view', len(bottleneck_view_dict['mid_level_view']), 'll_view', len(bottleneck_view_dict['low_level_view']))

('trange',) hl_view 1 ml_view 128 ll_view 128
('file_id',) hl_view 1 ml_view 128 ll_view 256
('proc_id',) hl_view 46 ml_view 92 ll_view 230
('trange', 'file_id') hl_view 1 ml_view 128 ll_view 256
('trange', 'proc_id') hl_view 46 ml_view 92 ll_view 230
('file_id', 'trange') hl_view 1 ml_view 128 ll_view 128
('file_id', 'proc_id') hl_view 46 ml_view 92 ll_view 184
('proc_id', 'trange') hl_view 1 ml_view 128 ll_view 128
('proc_id', 'file_id') hl_view 1 ml_view 128 ll_view 256
('trange', 'file_id', 'proc_id') hl_view 46 ml_view 92 ll_view 184
('trange', 'proc_id', 'file_id') hl_view 1 ml_view 128 ll_view 256
('file_id', 'trange', 'proc_id') hl_view 46 ml_view 92 ll_view 184
('file_id', 'proc_id', 'trange') hl_view 1 ml_view 128 ll_view 128
('proc_id', 'trange', 'file_id') hl_view 1 ml_view 128 ll_view 256
('proc_id', 'file_id', 'trange') hl_view 1 ml_view 128 ll_view 128
time: 3.85 s (started: 2023-03-08 09:20:57 -08:00)


In [40]:
bottleneck_views[('trange',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,random_time,data_count,data_size,data_time,size_min,size_max,duration_score,duration_cut,duration_pero,duration_perr
trange,,,,,,,,,,,,,,,,,,,,,
2,7.981181,256,1715918241792,128,0,128,1715918241792,0,0,7.981038,...,0,128,1715918241792,7.981038,0,13405611264,6,0.75,0.989582,0.013913


time: 33.9 ms (started: 2023-03-08 09:21:05 -08:00)


In [41]:
bottleneck_views[('file_id',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,random_time,data_count,data_size,data_time,size_min,size_max,duration_score,duration_cut,duration_pero,duration_perr
file_name,,,,,,,,,,,,,,,,,,,,,
/p/gpfs1/brainusr/datasets/jag/jag100K1vw_train.npy,8.047039,768,1715918243200,256,0,512,1715918243200,0,0,8.046688,...,0,256,1715918243200,8.046688,0,13405611264,6,0.75,0.997747,0.014028


time: 31.3 ms (started: 2023-03-08 09:21:12 -08:00)


In [42]:
bottleneck_views[('proc_id',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,random_time,data_count,data_size,data_time,size_min,size_max,duration_score,duration_cut,duration_pero,duration_perr
proc_name,,,,,,,,,,,,,,,,,,,,,
app1#localhost#104#35186539112624,7.190066,472,13405635851,5,0,467,13405635851,0,0,7.158589,...,0,5,13405635851,7.158589,0,13405611264,6,0.75,0.891492,0.012542
app1#localhost#105#35186539112624,7.198214,472,13405635851,5,0,467,13405635851,0,0,7.166389,...,0,5,13405635851,7.166389,0,13405611264,6,0.75,0.892502,0.012556
app1#localhost#110#35186539112624,7.626041,472,13405635851,5,0,467,13405635851,0,0,7.624501,...,0,5,13405635851,7.624501,0,13405611264,6,0.75,0.945548,0.013302
app1#localhost#111#35186539112624,7.653952,472,13405635851,5,0,467,13405635851,0,0,7.651655,...,0,5,13405635851,7.651655,0,13405611264,6,0.75,0.949009,0.013351
app1#localhost#114#35186539112624,7.312101,472,13405635851,5,0,467,13405635851,0,0,7.310972,...,0,5,13405635851,7.310972,0,13405611264,6,0.75,0.906623,0.012755
app1#localhost#115#35186539112624,7.313315,472,13405635851,5,0,467,13405635851,0,0,7.311906,...,0,5,13405635851,7.311906,0,13405611264,6,0.75,0.906773,0.012757
app1#localhost#122#35186539112624,7.237279,472,13405635851,5,0,467,13405635851,0,0,7.235481,...,0,5,13405635851,7.235481,0,13405611264,6,0.75,0.897346,0.012624
app1#localhost#123#35186539112624,7.247154,472,13405635851,5,0,467,13405635851,0,0,7.245365,...,0,5,13405635851,7.245365,0,13405611264,6,0.75,0.898570,0.012641
app1#localhost#124#35186539112624,6.092332,472,13405635851,5,0,467,13405635851,0,0,6.091143,...,0,5,13405635851,6.091143,0,13405611264,6,0.75,0.755384,0.010627


time: 56.5 ms (started: 2023-03-08 09:21:14 -08:00)


In [43]:
bottleneck_views[('proc_id',)]['mid_level_view'].compute()

duration_sum  index_count  \
proc_name                         trange                              
app1#localhost#104#35186539112624 1           0.085055          470   
                                  2           7.105011            2   
app1#localhost#105#35186539112624 1           0.085294          470   
                                  2           7.112920            2   
app1#localhost#110#35186539112624 1           0.092515          470   
...                                                ...          ...   
app1#localhost#9#35186539112624   2           7.492335            2   
app1#localhost#92#35186539112624  1           0.086812          470   
                                  2           7.271541            2   
app1#localhost#93#35186539112624  1           0.086971          470   
                                  2           7.243893            2   

                                             size_sum  read_count  \
proc_name                         trange                            
app1#localhost#104#35186539112624 1             24587           4   
                                  2       13405611264           1   
app1#localhost#105#35186539112624 1             24587           4   
                                  2       13405611264           1   
app1#localhost#110#35186539112624 1             24587           4   
...                                               ...         ...   
app1#localhost#9#35186539112624   2       13405611264           1   
app1#localhost#92#35186539112624  1             24587           4   
                                  2       13405611264           1   
app1#localhost#93#35186539112624  1             24587           4   
                                  2       13405611264           1   

                                          write_count  metadata_count  \
proc_name                         trange                                
app1#localhost#104#35186539112624 1                 0             466   
                                  2                 0               1   
app1#localhost#105#35186539112624 1                 0             466   
                                  2                 0               1   
app1#localhost#110#35186539112624 1                 0             466   
...                                               ...             ...   
app1#localhost#9#35186539112624   2                 0               1   
app1#localhost#92#35186539112624  1                 0             466   
                                  2                 0               1   
app1#localhost#93#35186539112624  1                 0             466   
                                  2                 0               1   

                                            read_size  write_size  \
proc_name                         trange                            
app1#localhost#104#35186539112624 1             24587           0   
                                  2       13405611264           0   
app1#localhost#105#35186539112624 1             24587           0   
                                  2       13405611264           0   
app1#localhost#110#35186539112624 1             24587           0   
...                                               ...         ...   
app1#localhost#9#35186539112624   2       13405611264           0   
app1#localhost#92#35186539112624  1             24587           0   
                                  2       13405611264           0   
app1#localhost#93#35186539112624  1             24587           0   
                                  2       13405611264           0   

                                          metadata_size  read_time  ...  \
proc_name                         trange                            ...   
app1#localhost#104#35186539112624 1                   0   0.053725  ...   
                                  2                   0   7.104864  ...   
app1#localhost#105#35186539112624 1                   0   0.053618  ... 

time: 40.4 ms (started: 2023-03-08 09:21:43 -08:00)


In [44]:
proc_id_bview = bottleneck_views[('proc_id',)]['low_level_view'].compute()
proc_id_bview = proc_id_bview.drop(columns=['file_id', 'proc_id'])
proc_id_bview

duration_sum  \
proc_name                         trange file_name                                                          
app1#localhost#104#35186539112624 1      /g/g92/haridev/.nccl.conf                               0.000006   
                                         /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...      0.066731   
                                         /p/gpfs1/haridev/software/lbann/applications/ph...      0.017526   
                                         UNKNOWN                                                 0.000793   
                                  2      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...      7.105011   
...                                                                                                   ...   
app1#localhost#93#35186539112624  1      /g/g92/haridev/.nccl.conf                               0.000006   
                                         /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...      0.068195   
                                         /p/gpfs1/haridev/software/lbann/applications/ph...      0.018083   
                                         UNKNOWN                                                 0.000687   
                                  2      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...      7.243893   

                                                                                             index_count  \
proc_name                         trange file_name                                                         
app1#localhost#104#35186539112624 1      /g/g92/haridev/.nccl.conf                                     1   
                                         /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...            4   
                                         /p/gpfs1/haridev/software/lbann/applications/ph...            5   
                                         UNKNOWN                                                     460   
                                  2      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...            2   
...                                                                                                  ...   
app1#localhost#93#35186539112624  1      /g/g92/haridev/.nccl.conf                                     1   
                                         /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...            4   
                                         /p/gpfs1/haridev/software/lbann/applications/ph...            5   
                                         UNKNOWN                                                     460   
                                  2      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...            2   

                                                                                                size_sum  \
proc_name                         trange file_name                                                         
app1#localhost#104#35186539112624 1      /g/g92/haridev/.nccl.conf                                     0   
                                         /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...           11   
                                         /p/gpfs1/haridev/software/lbann/applications/ph...        24576   
                                         UNKNOWN                                                       0   
                                  2      /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...  13405611264   
...                                                                                                  ...   
app1#localhost#93#35186539112624  1      /g/g92/haridev/.nccl.conf                                     0   
                                         /p/gpfs1/brainusr/datasets/jag/jag100K1vw_train...           11   
                                         /p/gpfs1/haridev/software/lbann/applications/ph...        24576   
                                         UNKNOWN                                  

time: 51.4 ms (started: 2023-03-08 09:21:45 -08:00)
